In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import API_KEY
gmaps.configure(api_key=API_KEY)

In [11]:
#Store Part I results (from WeatherPy) into DataFrame and Load the csv exported in Part I to a DataFrame.

weather_csv_file = "../WeatherPy/Output/weather_df.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()

,Unnamed: 0,City name,Country,Cloudiness,Dates,Humidities,Lats,Lngs,Max_temps,Wind_speeds
0,0,dhidhdhoo,MV,95,1631668720,76,6.8833,73.1000,83.23,15.37
1,1,sterling,US,20,1631668596,82,39.0062,-77.4286,80.89,4.61
2,2,asau,RO,7,1631668613,97,46.4333,26.4000,56.32,3.11
3,3,qaanaaq,GL,85,1631668721,91,77.4840,-69.3632,17.69,5.82
4,4,mataura,NZ,10,1631668721,49,-46.1927,168.8643,49.48,5.14



Out[2]:
City	Cloudiness	Country	Date	Humidity	Latitude	Longitude	Max Temp	Wind Speed
0	hasaki	75.0	JP	1.580018e+09	86.0	35.73	140.83	48.00	5.82
1	oktyabrskoye	0.0	RU	1.580018e+09	85.0	43.06	44.74	19.00	6.71
2	atuona	37.0	PF	1.580018e+09	76.0	-9.80	-139.03	81.00	14.47
3	rikitea	24.0	PF	1.580018e+09	78.0	-23.12	-134.97	78.73	11.10
4	bredasdorp	69.0	ZA	1.580018e+09	77.0	-34.53	20.04	63.00	12.75

In [12]:

# Humidity Heatmap
# Configure gmaps.
# Use the Lat and Lng as locations and Humidity as the weight.

# Configure gmaps with API key.
gmaps.configure(api_key=API_KEY)

# Also, handle NaN values
weather_df = weather_df.dropna()

# Convert Humidity to float and store
humidity = weather_df["Humidities"].astype(float)
humidity

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Lats", "Lngs"]].astype(float)
locations
# Create a humidity Heatmap layer
figure_layout = {
    'width': '1000px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}
fig = gmaps.figure(layout=figure_layout)
fig
# Add Heatmap layer to map.
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidities"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig



Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='1000px'))

In [13]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max_temps"] < 80) & (filtered_weather_df["Max_temps"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind_speeds"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,Unnamed: 0,City name,Country,Cloudiness,Dates,Humidities,Lats,Lngs,Max_temps,Wind_speeds
108,108,sinjar,IQ,0,1631668750,30,36.3209,41.8754,76.59,5.53
135,135,kashan,IR,0,1631668757,11,33.9831,51.4364,79.95,1.03
249,249,salinopolis,BR,0,1631668789,81,-0.6136,-47.3561,78.46,8.59
290,290,jalu,LY,0,1631668801,45,29.0331,21.5482,74.52,8.81
315,315,sinnamary,GF,0,1631668810,85,5.3833,-52.9500,78.64,7.67
367,367,belmonte,BR,0,1631668825,86,-15.8631,-38.8828,73.90,8.86
375,375,porto-vecchio,FR,0,1631668827,83,41.5910,9.2795,71.26,8.05
417,417,upata,VE,0,1631668840,89,8.0086,-62.3989,71.10,3.80
463,463,bastia,FR,0,1631668851,85,42.6667,9.3333,72.30,1.01
466,466,nanzhou,CN,0,1631668852,72,29.3596,112.4024,76.06,7.31


In [15]:
hotel_df =filtered_weather_df

hotel_df['Hotel Name'] = np.nan
hotel_df
target_search = "Hotel"
target_radius = 5000
target_type = "lodging"

idx = 0
for index, row in hotel_df.iterrows():
    # set up a parameters dictionary
    params = {
        "location": f"{row['Lats']}, {row['Lngs']}",
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": API_KEY
        
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    response_json = response.json()
    results = response_json['results']
    if len(results) > 0:
        name = response_json['results'][0]['name']
        hotel_df.iloc[idx, -1] = name
    idx = idx + 1

narrowed_city_df = hotel_df  
narrowed_city_df.dropna()


,Unnamed: 0,City name,Country,Cloudiness,Dates,Humidities,Lats,Lngs,Max_temps,Wind_speeds,Hotel Name
135,135,kashan,IR,0,1631668757,11,33.9831,51.4364,79.95,1.03,هتل
249,249,salinopolis,BR,0,1631668789,81,-0.6136,-47.3561,78.46,8.59,Hotel Privê do Atalaia
290,290,jalu,LY,0,1631668801,45,29.0331,21.5482,74.52,8.81,Jalu Hotel
315,315,sinnamary,GF,0,1631668810,85,5.3833,-52.9500,78.64,7.67,Hotel Du Fleuve
367,367,belmonte,BR,0,1631668825,86,-15.8631,-38.8828,73.90,8.86,Pousada Rio
375,375,porto-vecchio,FR,0,1631668827,83,41.5910,9.2795,71.26,8.05,Isulella Hotel & Restaurant
417,417,upata,VE,0,1631668840,89,8.0086,-62.3989,71.10,3.80,Hotel Emilia
463,463,bastia,FR,0,1631668851,85,42.6667,9.3333,72.30,1.01,La Dimora
466,466,nanzhou,CN,0,1631668852,72,29.3596,112.4024,76.06,7.31,Ramada Plaza by Wyndham Yiyang Nanxian
491,491,maceio,BR,0,1631668860,94,-9.6658,-35.7353,76.44,2.30,Jatiúca Hotel & Resort


In [16]:

# Assign the marker layer to a variable
locations = narrowed_city_df[["Lats", "Lngs"]]
markers = gmaps.marker_layer(locations)


In [17]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='1000px'))